In [16]:
boroughs = ["Camden", "Greenwich", "Hackney", 
            "Islington", "Lambeth", "Lewisham",
            "Southwark", "Wandsworth", "Westminster"]

In [10]:
boroughs = ["City_of_London"]

In [19]:
boroughs = ["Hammersmith_and_Fulham"]

In [23]:
boroughs = ["Tower_Hamlets"]

In [2]:
results_dir = 'all_original'

In [3]:
from transformers import AutoImageProcessor, TFSegformerForSemanticSegmentation
from PIL import Image
import requests
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import numpy as np

In [13]:
import pandas as pd

In [4]:
# SegFormer
image_processor = AutoImageProcessor.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512")
model = TFSegformerForSemanticSegmentation.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
C:\Users\win10\.conda\envs\yyjj125\lib\site-packages\transformers\models\segformer\image_processing_segformer.py:99: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


All model checkpoint layers were used when initializing TFSegformerForSemanticSegmentation.

All the layers of TFSegformerForSemanticSegmentation were initialized from the model checkpoint at nvidia/segformer-b0-finetuned-ade-512-512.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFSegformerForSemanticSegmentation for predictions without further training.


In [5]:
import cv2
import torch
import matplotlib.pyplot as plt

model_type = "DPT_Large" 

# MiDas
midas = torch.hub.load("intel-isl/MiDaS", model_type)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
midas.to(device)
midas.eval()

# Load the appropriate transformation function based on the model type
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform

Using cache found in C:\Users\win10/.cache\torch\hub\intel-isl_MiDaS_master
Using cache found in C:\Users\win10/.cache\torch\hub\intel-isl_MiDaS_master


In [6]:
width=640
height=640
tree = 4
grass = 9
shrub=17
total=width*height

In [24]:
# iterate borough
for borough in boroughs:
    image_dir = borough
    results = []
    if not os.path.isdir(image_dir):
        print(f'no: {image_dir}')
        continue
    image_files = os.listdir(image_dir)
    for image_file in image_files:.
        image_path = os.path.join(image_dir, image_file)
        img = Image.open(image_path)
        img_np = np.array(img)
        inputs = image_processor(images=img, return_tensors="tf")
        outputs = model(**inputs, training=False)
        input_batch = transform(img_np).to(device) 
        with torch.no_grad():
            prediction = midas(input_batch)
            prediction = torch.nn.functional.interpolate(
                prediction.unsqueeze(1),
                size=img.size,
                mode="bicubic",
                align_corners=False,
            ).squeeze()      
        depth_output = prediction.cpu().numpy()
        label_map = tf.argmax(outputs.logits, axis=1).numpy()
        label_map_first = label_map[0]
        label_map_resized = cv2.resize(label_map_first, (prediction.shape[1], prediction.shape[0]), interpolation=cv2.INTER_NEAREST)
        for label, label_name in zip([tree, grass, shrub], ['tree', 'grass', 'shrub']):
            label_map_binary = (label_map_resized == label)
            num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(label_map_binary.astype('uint8'), connectivity=8)
            instance_counter = 0
            for i in range(1, num_labels):
                instance_pixels = stats[i, -1]
                if instance_pixels > 1250:
                    instance_percentage = (instance_pixels / total) * 100
                    instance_depth = depth_output[labels == i].mean()
                    instance_counter += 1  # Increment instance counter
                    filename_parts = image_file.split('_')
                    point_id = int(filename_parts[3])
                    heading = int(filename_parts[4].split('.')[0])
                    results.append({
                        'PointID': point_id,
                        'heading': heading,
                        'category': label_name,
                        'object': instance_counter,
                        'percentage of green': instance_percentage,
                        'average depth': instance_depth
                    })

    # make df for results
    df_results = pd.DataFrame(results)
    # save to CSV
    df_results.to_csv(os.path.join(results_dir, f"{borough}_original.csv"), index=False)

In [ ]:
# I acknowledge the use of ChatGPT 4.0 (Open AI, https://chat.openai.com) to debug.